In [1]:
# ============================================================
# PredictCycleLife_XGBoost.ipynb
# Amaç: features_early_cycles.csv -> XGBoost ile ömür tahmini
# ============================================================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

from xgboost import XGBRegressor

# ------------------------------------------------------------
# 1) Veriyi yükle
# ------------------------------------------------------------
df = pd.read_csv("../features_early_cycles.csv")
print("Toplam satır:", len(df))
print(df.head())

# ------------------------------------------------------------
# 2) Hedef değişkeni (cycle_life) temizle
# ------------------------------------------------------------
df = df.copy()

df["cycle_life"] = (
    df["cycle_life"]
    .astype(str)
    .str.strip()
    .str.replace(r"[\[\]]", "", regex=True)  # varsa [] kaldır
)
df["cycle_life"] = pd.to_numeric(df["cycle_life"], errors="coerce")

print("cycle_life içindeki NaN sayısı:", df["cycle_life"].isna().sum())

# ------------------------------------------------------------
# 3) Feature kolonlarını belirle
# ------------------------------------------------------------

candidate_features = [
    "Qd_mean", "Qd_std", "IR_mean", "IR_std", "Tavg_mean", "dQd_slope",
    "dqdv_peak_first", "dqdv_peak_last", "dqdv_peak_delta",
    "dqdv_peak_mean", "dqdv_peak_std", "dqdv_peak_slope",
    "dqdv_area_first", "dqdv_area_last", "dqdv_area_delta",
    "dqdv_area_mean", "dqdv_area_std", "dqdv_area_slope",
]

feature_cols = [c for c in candidate_features if c in df.columns]
print("\nKullanılacak feature kolonları:")
print(feature_cols)

# Feature + cycle_life içinde NaN olan satırları at
df = df.dropna(subset=["cycle_life"] + feature_cols)
print("\nTemizlik sonrası satır sayısı:", len(df))

# ------------------------------------------------------------
# 4) Her n_cycles için XGBoost modeli eğit
# ------------------------------------------------------------

results = {}

for n_cycles in [25, 50, 100]:
    print(f"\n{'='*60}")
    print(f"n_cycles = {n_cycles} için XGBoost modeli")
    print(f"{'='*60}")
    
    df_subset = df[df["n_cycles"] == n_cycles].copy()
    print("Bu n_cycles için satır sayısı:", len(df_subset))
    
    if len(df_subset) < 10:
        print("⚠ Yeterli veri yok, bu n_cycles atlanıyor.")
        continue
    
    X = df_subset[feature_cols]
    y = df_subset["cycle_life"]
    
    # Train-test ayır
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # --------------------------------------------------------
    # XGBoost modeli
    # --------------------------------------------------------
    model = XGBRegressor(
        n_estimators=800,
        learning_rate=0.03,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=2.0,
        reg_alpha=1.0,
        random_state=42,
        n_jobs=-1,
        tree_method="hist",   # GPU yoksa da hızlıdır
    )
    
    # Modeli eğit
    model.fit(X_train, y_train)
    
    # Tahmin
    y_pred = model.predict(X_test)
    
    # Performans metrikleri
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"MAE: {mae:.2f}")
    print(f"R² : {r2:.4f}")
    
    # Özellik önemleri
    importances = pd.Series(model.feature_importances_, index=feature_cols)
    importances = importances.sort_values(ascending=False)
    
    print("\nÖzellik önemleri (XGBoost):")
    for feat, imp in importances.items():
        print(f"  {feat:20s}: {imp:.4f}")
    
    results[n_cycles] = {"MAE": mae, "R2": r2}

# ------------------------------------------------------------
# 5) Özet tablo
# ------------------------------------------------------------

print("\n\n=== XGBOOST ÖZET SONUÇLAR ===")
for n, res in results.items():
    print(f"n_cycles = {n:3d} → MAE = {res['MAE']:.2f}, R² = {res['R2']:.4f}")


Toplam satır: 117
  cell_id  n_cycles  cycle_life   Qd_mean    Qd_std   IR_mean    IR_std  \
0    b1c0        25       477.0  1.104835  0.001730  0.017266  0.000128   
1    b1c0        50       477.0  1.105175  0.002964  0.017289  0.000118   
2    b1c0       100       477.0  1.105038  0.003119  0.017338  0.000122   
3    b1c1        25       491.0  1.107000  0.001108  0.017764  0.000211   
4    b1c1        50       491.0  1.106932  0.001209  0.017645  0.000195   

   Tavg_mean  dQd_slope  dqdv_peak_first  ...  dqdv_area_delta  \
0  31.822145   0.000161              0.0  ...        -3.478434   
1  31.804072   0.000022              0.0  ...        36.278293   
2  31.807497   0.000016              0.0  ...        48.199909   
3  33.542142   0.000030              0.0  ...         3.463564   
4  33.605174   0.000016              0.0  ...        11.476454   

   dqdv_area_mean  dqdv_area_std  dqdv_area_slope  dqdv_peakpos_first  \
0     -748.333411       8.318482         0.077890            